<a href="https://colab.research.google.com/github/saishshinde15/Langchain_RAG_BOT/blob/main/KG_RAG_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [2]:
# Neo4j and py2neo for Knowledge Graph handling
!pip install neo4j
!pip install py2neo

# LangChain and related libraries
!pip install langchain
!pip install openai  # For OpenAI LLM support in LangChain
!pip install sentence-transformers  # For Hugging Face embeddings
!pip install faiss-cpu  # FAISS library for vector storage
!pip install graphviz  # For generating ER diagrams if required

# Hugging Face Transformers for HuggingFaceEmbeddings
!pip install transformers

# Optionally, install additional utilities if you need them
!apt-get install -y graphviz


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 42.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [3]:
%pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 6.4 MB/s eta 0:00:00


In [4]:
import os
from py2neo import Graph
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import WebBaseLoader
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA
#from langchain.retrievers import VectorstoreRetriever
from langchain_core.vectorstores import VectorStoreRetriever

In [5]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [6]:
llm=ChatGroq(model='llama-3.1-8b-instant',temperature=0.5)

In [7]:
from google.colab import userdata
password=userdata.get('neo4j')

In [8]:
NEO4J_URI = "neo4j+s://1837079e.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = password

graph = Graph(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Test the connection
try:
    graph.run("MATCH (n) RETURN count(n)")
    print("Successfully connected to Neo4j!")
except Exception as e:
    print(f"Failed to connect to Neo4j: {e}")

Successfully connected to Neo4j!


In [9]:
def classify_cybersecurity_query(query):
    """
    Classifies a cybersecurity-related query using a pre-defined knowledge graph.

    Args:
        query: The input query string.

    Returns:
        A string representing the classification of the query.
        Returns an error message if the query classification fails.
    """
    try:
        # Example classification logic using a simple keyword-based approach
        #  (replace with your actual knowledge graph query)
        query = query.lower()

        if "malware" in query or "virus" in query or "ransomware" in query:
            return "Malware Analysis"
        elif "phishing" in query or "social engineering" in query:
            return "Social Engineering and Phishing"
        elif "network security" in query or "firewall" in query or "intrusion detection" in query:
            return "Network Security"
        elif "vulnerability" in query or "penetration testing" in query:
            return "Vulnerability Management"
        elif "incident response" in query or "security breach" in query:
            return "Incident Response"
        else:
            # Use Neo4j to search for related concepts in the knowledge graph
            # Construct a Cypher query based on the user's input to find relevant concepts or categories
            cypher_query = f"""
                MATCH (c:CybersecurityConcept)
                WHERE toLower(c.name) CONTAINS toLower("{query}")
                RETURN c.category
            """
            result = graph.run(cypher_query).data()

            # Extract the category from the query result
            if result:
                return result[0].get("c.category", "Unknown")
            else:
                return "Unknown Category"

    except Exception as e:
        return f"Error classifying query: {e}"

In [10]:
# Change accordingly to the data in neo4js
def query_knowledge_graph(query):
    """Query the cybersecurity knowledge graph for entities and relationships."""
    cypher_query = f"""
    MATCH (threat:ThreatActor)-[r:EXPLOITS]->(vulnerability:Vulnerability)
    WHERE threat.name CONTAINS '{query}' OR vulnerability.name CONTAINS '{query}'
    RETURN threat.name AS ThreatActor, r AS Exploit, vulnerability.name AS Vulnerability LIMIT 10
    """
    results = graph.run(cypher_query).data()

    if results:
        knowledge = []
        for result in results:
            knowledge.append(f"{result['ThreatActor']} -[EXPLOITS]→ {result['Vulnerability']}")
        return knowledge
    else:
        return ["No relevant threats or vulnerabilities found in the knowledge graph."]

In [12]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter

def fetch_and_split(urls):
    loader = WebBaseLoader(urls)
    data = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
    docs = text_splitter.split_documents(data)
    return docs

# Example usage
urls = ["https://www.simplilearn.com/tutorials/cyber-security-tutorial/types-of-cyber-attacks", "https://cheatsheetseries.owasp.org/cheatsheets/SQL_Injection_Prevention_Cheat_Sheet.html"] # Replace with your URLs
documents = fetch_and_split(urls)

for doc in documents:
    print(doc.page_content[:50]) # Print first 50 chars of each chunk

Types of Cyber Attacks You Should Be Aware of in 2
All CoursesAll CoursesReviewsFor BusinessResources
© 2009 -2024- Simplilearn Americas Inc.Follow us!R
SQL Injection Prevention - OWASP Cheat Sheet Serie
Microservices Security
  

    Microservices based
Related Articles
    


    Secrets Management
  

Details Of Least Privilege When Developing
    

 
Option 1: Use of Prepared Statements (with Paramet
Safe C# .NET Prepared Statement Example¶
In .NET, 
Other Examples of Safe Prepared Statements¶
If you
Safe VB .NET Stored Procedure Example¶
The followi
Defense Option 3: Allow-list Input Validation¶
If 
Safest Use Of Dynamic SQL Generation (DISCOURAGED)
Any time user input can be converted to a non-Stri
Least Privilege¶
To minimize the potential damage 
SQL Injection Cheat Sheet
Bypassing WAF's with SQL


In [13]:
def create_faiss_vector_store(docs):
    """
    Creates a FAISS vector store from a list of documents using Hugging Face embeddings.
    """
    try:
        embeddings = HuggingFaceEmbeddings()
        vectorstore = FAISS.from_documents(docs, embeddings)
        return vectorstore
    except Exception as e:
        print(f"Error creating FAISS vector store: {e}")
        return None

In [14]:
def hybrid_retrieval(query):
    """
    Combines knowledge graph and document retrieval to answer a cybersecurity query.
    """
    # 1. Classify the query using the knowledge graph
    classification = classify_cybersecurity_query(query)
    print(f"Query Classification: {classification}")

    # 2. Query the knowledge graph based on the query
    kg_results = query_knowledge_graph(query)
    print(f"Knowledge Graph Results:\n{kg_results}")

    # 3. Perform document retrieval using FAISS
    vectorstore = create_faiss_vector_store(documents)
    if vectorstore:
        retriever = VectorStoreRetriever(vectorstore=vectorstore)
        qa = RetrievalQA.from_chain_type(
            llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True
        )
        document_results = qa({"query": query})
        print(f"Document Retrieval Results:\n{document_results['result']}")
        #print(f"\nSource Documents:\n{document_results['source_documents']}")

    else:
        print("Failed to create FAISS vector store. Skipping document retrieval.")

    #Combine results and return
    combined_results = {
        "classification": classification,
        "knowledge_graph": kg_results,
        "document_retrieval": document_results['result'] if vectorstore else "Unavailable"
    }
    return combined_results

In [15]:
def generate_response(user_query):
    """
    Generates a comprehensive response to a user query using a hybrid approach
    combining knowledge graph and document retrieval.
    """
    try:
        response = hybrid_retrieval(user_query)
        return response
    except Exception as e:
        return f"An error occurred: {e}"

In [17]:
def user_feedback(query, response, feedback):
    """
    Collects user feedback on the generated response.

    Args:
        query: The original user query.
        response: The generated response.
        feedback: The user's feedback (e.g., "good", "bad", "needs improvement").
    """
    try:
        # Example of storing feedback in a text file
        with open("user_feedback.txt", "a") as f:  # Example using a text file
            f.write(f"Query: {query}\n")
            f.write(f"Response: {response}\n")
            f.write(f"Feedback: {feedback}\n\n")

        print("Feedback recorded successfully!")

    except Exception as e:
        print(f"Error recording feedback: {e}")

In [18]:
# Main Function
def main():
    """Main function to handle user queries and feedback."""
    while True:
        user_query = input("Enter your cybersecurity query (or type 'exit' to quit): ")
        if user_query.lower() == "exit":
            break

        response = generate_response(user_query)
        print(f"\nResponse:\n{response}")

        feedback = input("Enter your feedback (good, bad, needs improvement, or skip): ").lower()
        if feedback != "skip":
            user_feedback(user_query, response, feedback)

if __name__ == "__main__":
    main()

Enter your cybersecurity query (or type 'exit' to quit): what is sql injection attack
Query Classification: Unknown Category
Knowledge Graph Results:
['No relevant threats or vulnerabilities found in the knowledge graph.']


<ipython-input-13-db9d15b26f86>:6: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


Document Retrieval Results:
A SQL injection attack is a type of cyber attack where an attacker injects malicious SQL code into a web application's database to extract or modify sensitive data, or to disrupt the application's functionality.

Here's a simplified explanation of how a SQL injection attack works:

1. **Vulnerable application**: A web application uses user input to construct SQL queries, without properly validating or sanitizing the input. This allows an attacker to inject malicious SQL code.
2. **Malicious input**: An attacker enters malicious SQL code into the application's input field, such as a username or password field.
3. **SQL query construction**: The application constructs a SQL query using the user input, which includes the malicious SQL code.
4. **SQL injection**: The malicious SQL code is executed by the database, allowing the attacker to extract or modify sensitive data, or to disrupt the application's functionality.

Types of SQL injection attacks:

1. **Class

<ipython-input-13-db9d15b26f86>:6: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


Document Retrieval Results:
An insider threat is a type of cyber threat that originates from within an organization, typically from an authorized individual who has access to sensitive information or systems. Insider threats can be intentional or unintentional, and they can cause significant damage to an organization's security, reputation, and bottom line.

Types of Insider Threats:

1. **Malicious Insider**: A malicious insider is an individual who intentionally causes harm to the organization by stealing sensitive information, sabotaging systems, or disrupting operations.
2. **Negligent Insider**: A negligent insider is an individual who unintentionally causes harm to the organization by failing to follow security policies, procedures, or best practices.
3. **Accidental Insider**: An accidental insider is an individual who unintentionally causes harm to the organization by making a mistake or clicking on a malicious link.
4. **Compromised Insider**: A compromised insider is an indiv